In [ ]:
# The preliminary notebook for testing/developing wrsamp. Will delete before merging. 

wrsamp calls wrheader with sig. wrheader calls setfields, and checkfields. 


setfields() - fills in fields of a wfdbrecord object. Fields that it can fill in include essential fields, and dependencies.

It will NOT overwrite any user defined fields, aka fields != None. 





Fields that depend on other fields:
    - 

Perhaps easiest solution is to specify a default order of filling in fields. Need to separate by record and signal? Nope, just put the record ones before the signal ones!!!!

In [12]:
class WFDBrecord():
    
    # Constructor
    def __init__(self, signals=None, recordname=None, nsig=None, 
                 fs=None, counterfreq=None, basecounter=None, 
                 siglen=None, basetime=None, basedate=None, 
                 filename=None, fmt=None, sampsperframe=None, 
                 skew=None, byteoffset=None, adcgain=None, 
                 baseline=None, units=None, adcres=None, 
                 adczero=None, initvalue=None, checksum=None, 
                 blocksize=None, signame=None, comments=None):
        
        # Note the lack of 'nseg' field. Single segment records cannot have this field. Even nseg = 1 makes 
        # the header a multi-segment header. 
        
        self.signals = signals
        
        self.recordname=recordname
        self.nsig=nsig
        self.fs=fs
        self.counterfreq=counterfreq
        self.basecounter=basecounter
        self.siglen=siglen
        self.basetime=basetime
        self.basedate=basedate
        
        self.filename=filename
        self.fmt=fmt
        self.sampsperframe=sampsperframe
        self.skew=skew
        self.byteoffset=byteoffset
        self.adcgain=adcgain
        self.baseline=baseline
        self.units=units
        self.adcres=adcres
        self.adczero=adczero
        self.initvalue=initvalue
        self.checksum=checksum
        self.blocksize=blocksize
        self.signame=signame
        
        self.comments=comments

In [35]:
import numpy as np
# The specifications of a WFDB field
# 0. type
# 1. delimiter
# 2. dependency
# 3. required to write? (nseg starts as F but will be set by wrmultiheader and wrmultiseg to True.)
# 4. default fill function available? 0-2. 
class WFDBfieldspecs():
    
    def __init__(self, speclist):
    
        # Data types the field can take
        self.allowedtypes = speclist[0]
        
        # The text delimiter that preceeds the field
        self.delimiter = speclist[1]
        
        # The required/dependent field which must also be present
        self.dependency = speclist[2]
        
        # Whether the field is mandatory for writing a header (WFDB requirements + extra rules enforced by this library).
        # Being required for writing is not the same as the user having to specify via wrsamp/wrhea. These functions can set defaults.
        self.write_req = speclist[3]
        
        # Whether there is a default value for this field that can be inferred or calculated, and also whether ther setfield function will actually set the field. 
        # 0 = no, 1 = yes but only if signals is present, 2 = yes without needing signals (although it may still use it).  
        # Watch out for: adcgain, baseline. Set them to 1 but have a check statement in the function. Should be 0 if isdigital.
        # sampsperframe, skew, and byteoffset do have defaults but the setfields will not return anything because we do not want to write anything. So value = 0. Conversely when reading, these fields will be left as None if not present. 
        # This field is used to see if we can call the 'setfield' function on that field.
        self.has_write_default = speclist[4]
         

# The signal field
signalspecs = OrderedDict([('signal', WFDBfieldspecs([[np.ndarray], '', None, False, 2]))])

# The segment field
segmentspecs = OrderedDict([('segment', WFDBfieldspecs([[list], '', None, True, 2]))])

# Record specification fields            
recfieldspecs = OrderedDict([('recordname', WFDBfieldspecs([[str], '', None, True, 0])),
                         ('nseg', WFDBfieldspecs([[int], '/', 'recordname', True, 0])), # Essential for multi but illegal for single.
                         ('nsig', WFDBfieldspecs([[int], ' ', 'recordname', True, 1])),
                         ('fs', WFDBfieldspecs([[int, float], ' ', 'nsig', True, 0])),
                         ('counterfreq', WFDBfieldspecs([[int, float], '/', 'fs', False, 0])),
                         ('basecounter', WFDBfieldspecs([[int, float], '(', 'counterfreq', False, 0])),
                         ('siglen', WFDBfieldspecs([[int], ' ', 'fs', True, 1])),
                         ('basetime', WFDBfieldspecs([[str], ' ', 'siglen', False, 2])),
                         ('basedate', WFDBfieldspecs([[str], ' ', 'basetime', False, 0]))])
# Signal specification fields 
sigfieldspecs = OrderedDict([('filename', WFDBfieldspecs([[str], '', None, True, 2])),
                         ('fmt', WFDBfieldspecs([[int, str], ' ', 'filename', True, 2])),
                         ('sampsperframe', WFDBfieldspecs([[int], 'x', 'fmt', False, 0])),
                         ('skew', WFDBfieldspecs([[int], ':', 'fmt', False, 0])),
                         ('byteoffset', WFDBfieldspecs([[int], '+', 'fmt', False, 0])),
                         ('adcgain', WFDBfieldspecs([[int], ' ', 'fmt', True, 1])),
                         ('baseline', WFDBfieldspecs([[int], '(', 'adcgain', True, 1])),
                         ('units', WFDBfieldspecs([[str], '/', 'adcgain', True, 2])),
                         ('adcres', WFDBfieldspecs([[int], ' ', 'adcgain', False, 2])),
                         ('adczero', WFDBfieldspecs([[int], ' ', 'adcres', False, 2])),
                         ('initvalue', WFDBfieldspecs([[int], ' ', 'adczero', False, 1])),
                         ('checksum', WFDBfieldspecs([[int], ' ', 'initvalue', False, 1])),
                         ('blocksize', WFDBfieldspecs([[int], ' ', 'checksum', False, 2])),
                         ('signame', WFDBfieldspecs([[str], ' ', 'blocksize', False, 1]))])
    
# Segment specification fields
segfieldspecs = OrderedDict([('segname', WFDBfieldspecs([[str], '', None, True, 0])),
                         ('seglen', WFDBfieldspecs([[int], ' ', 'segname', True, 0]))])
# Comment field
comfieldspecs = OrderedDict([('comments', WFDBfieldspecs([[int], '', None, False, False]))])


singlefieldspeclist = [signalspecs.copy(), recfieldspecs.copy(), sigfieldspecs.copy(), comfieldspecs.copy()]
del(singlefieldspeclist[1]['nseg'])
multifieldspeclist = [segmentspecs.copy(), recfieldspecs.copy(), segfieldspecs.copy(), comfieldspecs.copy()]

In [52]:
fieldspeclist = singlefieldspeclist
for f in OrderedDict(reversed(list(fieldspeclist[1].items()))):
    print(f)

basedate
basetime
siglen
basecounter
counterfreq
fs
nsig
recordname


In [56]:
a=[1,2,3]
b=a
print(a is b)
b=[2,3,4]
a is b

True


False

In [49]:
a=[1,2,3]
b=a.copy()
b is a

False

In [30]:
rev_rfields=OrderedDict(reversed(list(WFDBfieldlist[0].items())))

In [32]:
rev_rfields['basedate'].write_req

False

In [51]:
a=None
if a and a>5:
    print('hi')

TypeError: unorderable types: NoneType() > int()

In [1]:
from testmodule import bread

In [4]:
panini=bread()
panini.getcals(3)

300

In [6]:
x='hello'
if not x:
    print('hi')

In [2]:
# pseudocode.
# setfield and checkfield should NOT be visible to the user. Fuck them, don't want them shitting it up.
# It makes the code 100x harder to write. 
# We don't even need to remove foreign fields it seems. Who gives a shit? We don't even use them anyway. 

def wrsamp(record):
    
    record.checkfields([signals, physical])
    record.wrheader()
    

def wrheader(record):
    
    writefields = record.getwritefields() # Get all the fields needed to write the record
    
    record.checkfields(writefields, setmissing = 1, fieldcheckorder = ()) # 
    
    record.wrheaderfile()
    
    
# Check each field in the list of fields. If setmissing is true, try to set the field if it == None. 
# The checking order is specified by fieldcheckorder. Hopefully via the checking order, there will be no 
# Fieldcheckorder starts from record line and moves onto signal/segment line. 
def checkfields(self, fieldstocheck, setmissing, fieldcheckorder = []):
        
    # Check fields in order because the validity of some may be dependent on others. 
    # In addition, the setvalue of some may depend on 
    for f in fieldcheckorder:
        if f in fieldstocheck:
            checkfield(self, f, setmissing)
                
                
# Check whether a field is valid. If the 'setfield' option is 1, the function will try to set a default if possible. 
# Checkfield calls setfield when setmissing == 1, and setfield may call checkfield (on another field) with setmissing = 0.
def checkfield(self, field, setmissing=0):
    
    errormsg = None
    
    # Try to set the value of the missing field if specified. 
    # Note, this may do nothing. 
    if setvalue and self.field==None:
        setfield(self, field)
        
    # If the field is still missing, trigger an error
    if self.field == None:
        sys.exit("Missing field required: "+field)
    
    if field==a:
            
    elif field==b:
           
        checkfield(e)
        
            
    elif field==b:    
    elif field==b:
    
    return errormsg
# The reason why this checkfield method returns an error message rather than directly triggering an error is because
# checkfield is called by checkfields on fields we need to have, but it is also called by setfield to check other
# fields needed to set default values. So in the first case, we will exit using the error message. In the second case
# we will ignore the error message and continue without being able to set the field. 


    
# Set the field if possible. Otherwise do nothing and return (no error message).
# This method WILL overwrite fields already set. Key is to not call it on fields already set to avoid this. 
# Although for gain and baseline when physical == 1, it will overwrite anyway. 
# Not all fields have defaults, and some fields' defaults may rely on other fields being present and valid. 
def setfield(self, field):
    
    if field == 'nsig':
        if not checkfield(self, 'signals', 0):
            self.nsig = np.shape(self.signals)[1]
    elif field == 'siglen':
        if not checkfield(self, 'signals', 0):
            self.nsig = np.shape(self.signals)[1]
    elif field == 'basetime':
        self.basetime = '00:00:00'
        
        
    # By the time these fields are called to be set, nsig should be set already. 
    # This method will only suggest single dat records. 
    elif field == 'filename':
        self.filename = self.nsig*[self.recordname+'.dat']
    
    
    # fmt, adcgain, and baseline go together. 
    elif field == 'fmt':
        # This method will 
    
    elif field == 'adcgain':
        
        # Cannot infer a gain or baseline if the signal is already digital.
        if self.physical == 1:
            # Calculate and set the gain and baseline.  
            if not checkfield(self, 'signals', 0):
                [gain, baseline] = 
            
    elif field == 'baseline':
        if self.physical == 1:
            # Calculate and set the gain and baseline.  
            if not checkfield(self, 'signals', 0):
                [gain, baseline] = 

    elif field == 'adcres':
        if not checkfield(self, 'fmt', 0):
            if self.fmt in ['8', '80']:
                self.adcres = self.nsig*[8]
            elif self.fmt == '212':
                self.adcres = self.nsig*[12]
            elif self.fmt == '16':
                self.adcres = self.nsig*[16]
            elif self.fmt = '24':
                self.adcres = self.nsig*[24]
            elif self.fmt = '32':
                self.adcres = self.nsig*[32]
    elif field == 'adczero':
        if not checkfield(self, 'nsig', 0):
            self.adczero = self.nsig*[0]
    elif field == 'initvalue':   
        if not checkfield(self, 'signals', 0):
            self.initvalue = self.signals[0,:]
    elif field == 'checksum':
        if not checkfield(self, 'signals', 0):
            self.initvalue = self.computechecksum()
    elif field == 'blocksize':
        self.blocksize = 0
    
    # To do for multirecord (or maybe it's better not in this function): elif field == 'seglen':
        

# Record specification fields            
recfieldspecs = OrderedDict([('recordname', WFDBfield([[str], '', None, True, 0])),
                         ('nseg', WFDBfield([[int], '/', 'recordname', True, 0])), # Essential for multi but not present in single.
                         ('nsig', WFDBfield([[int], ' ', 'recordname', True, 1])),
                         ('fs', WFDBfield([[int, float], ' ', 'nsig', True, 0])),
                         ('counterfreq', WFDBfield([[int, float], '/', 'fs', False, 0])),
                         ('basecounter', WFDBfield([[int, float], '(', 'counterfreq', False, 0])),
                         ('siglen', WFDBfield([[int], ' ', 'fs', True, 1])),
                         ('basetime', WFDBfield([[str], ' ', 'siglen', False, 2])),
                         ('basedate', WFDBfield([[str], ' ', 'basetime', False, 0]))])
# Signal specification fields 
sigfieldspecs = OrderedDict([('filename', WFDBfield([[str], '', None, True, 2])),
                         ('fmt', WFDBfield([[int, str], ' ', 'filename', True, 2])),
                         ('sampsperframe', WFDBfield([[int], 'x', 'fmt', False, 0])),
                         ('skew', WFDBfield([[int], ':', 'fmt', False, 0])),
                         ('byteoffset', WFDBfield([[int], '+', 'fmt', False, 0])),
                         ('adcgain', WFDBfield([[int], ' ', 'fmt', True, 1])),
                         ('baseline', WFDBfield([[int], '(', 'adcgain', True, 1])),
                         ('units', WFDBfield([[str], '/', 'adcgain', True, 2])),
                         ('adcres', WFDBfield([[int], ' ', 'adcgain', False, 2])),
                         ('adczero', WFDBfield([[int], ' ', 'adcres', False, 2])),
                         ('initvalue', WFDBfield([[int], ' ', 'adczero', False, 1])),
                         ('checksum', WFDBfield([[int], ' ', 'initvalue', False, 1])),
                         ('blocksize', WFDBfield([[int], ' ', 'checksum', False, 2])),
                         ('signame', WFDBfield([[str], ' ', 'blocksize', False, 1]))])
    
# Segment specification fields
segfieldspecs = OrderedDict([('segname', WFDBfield([[str], '', None, True, 0])),
                         ('seglen', WFDBfield([[int], ' ', 'segname', True, 0]))])





IndentationError: expected an indented block (<ipython-input-2-39f8aa9a3ede>, line 50)